In [1]:
import math

from einops import rearrange
from torch import Tensor, nn

from zeta.nn.modules.simple_mamba import MambaBlock


class UMambaBlock_2D(nn.Module):
    """
    UMambaBlock_2D is a 4d Mamba block that can be used as a building block for a 4d visual model
    From the paper: https://arxiv.org/pdf/2401.04722.pdf

    Args:
        dim (int): The input dimension.
        dim_inner (Optional[int]): The inner dimension. If not provided, it is set to dim * expand.
        depth (int): The depth of the Mamba block.
        d_state (int): The state dimension. Default is 16.
        expand (int): The expansion factor. Default is 2.
        dt_rank (Union[int, str]): The rank of the temporal difference (Δ) tensor. Default is "auto".
        d_conv (int): The dimension of the convolutional kernel. Default is 4.
        conv_bias (bool): Whether to include bias in the convolutional layer. Default is True.
        bias (bool): Whether to include bias in the linear layers. Default is False.

    Examples::
        import torch
        # img:         B, C, H, W
        img_tensor = torch.randn(1, 64, 10, 10)

        # Initialize Mamba block
        block = UMambaBlock(dim=64, depth=1)

        # Forward pass
        y = block(img_tensor)
        print(y.shape)

    """

    def __init__(
        self,
        dim: int = None,
        depth: int = 5,
        d_state: int = 16,
        expand: int = 2,
        d_conv: int = 4,
        conv_bias: bool = True,
        bias: bool = False,
    ):
        super().__init__()
        self.dim = dim
        self.depth = depth
        self.d_state = d_state
        self.expand = expand
        self.d_conv = d_conv
        self.conv_bias = conv_bias
        self.bias = bias

        # If dt_rank is not provided, set it to ceil(dim / d_state)
        dt_rank = math.ceil(self.dim / 16)
        self.dt_rank = dt_rank

        # If dim_inner is not provided, set it to dim * expand
        dim_inner = dim * expand
        self.dim_inner = dim_inner

        # If dim_inner is not provided, set it to dim * expand
        self.in_proj = nn.Linear(dim, dim_inner, bias=False)
        self.out_proj = nn.Linear(dim_inner, dim, bias=False)

        # Implement 2d convolutional layer
        # 3D depthwise convolution
        self.conv1 = nn.Conv2d(
            in_channels=dim,
            out_channels=dim_inner,
            kernel_size=3,
            padding=1,
            stride=1,
        )

        self.conv2 = nn.Conv2d(
            in_channels=dim_inner,
            out_channels=dim,
            kernel_size=3,
            padding=1,
            stride=1,
        )

        # Init instance normalization
        self.instance_norm = nn.InstanceNorm2d(dim)
        self.instance_norm2 = nn.InstanceNorm2d(dim_inner)

        # Leaky RELU
        self.leaky_relu = nn.LeakyReLU()

        # Layernorm
        self.norm = nn.LayerNorm(dim)

        # Mamba block
        self.mamba = MambaBlock(
            dim=dim,
            depth=depth,
            d_state=d_state,
            expand=expand,
            d_conv=d_conv,
            conv_bias=conv_bias,
            bias=bias,
        )

    def forward(self, x: Tensor):
        """
        B, C, H, W
        """
        b, c, h, w = x.shape
        input = x
        print(f"Input shape: {x.shape}")

        # Apply convolution
        x = self.conv1(x)
        print(f"Conv1 shape: {x.shape}")

        # # Instance Normalization
        x = self.instance_norm(x) + self.leaky_relu(x)
        print(f"Instance Norm shape: {x.shape}")

        # TODO: Add another residual connection here

        x = self.conv2(x)

        x = self.instance_norm(x) + self.leaky_relu(x)

        x = x + input

        # # Flatten to B, L, C
        x = rearrange(x, "b c h w -> b (h w) c")
        print(f"Faltten shape: {x.shape}")
        x = self.norm(x)

        # Maybe use a mamba block here then reshape back to B, C, H, W, D
        x = self.mamba(x)

        # Reshape back to B, C, H, W, D
        x = rearrange(x, "b (h w) c -> b c h w", h=h, w=w)

        return x

In [4]:
import math

from einops import rearrange
from torch import Tensor, nn
import torch

from zeta.nn.modules.simple_mamba import MambaBlock


img_tensor = torch.randn(2, 64, 96, 96).cuda()

# Initialize Mamba block
block = UMambaBlock_2D(dim=64, depth=5).cuda()

# Forward pass
y = block(img_tensor)
print(y.shape, y.device)

Input shape: torch.Size([2, 64, 96, 96])
Conv1 shape: torch.Size([2, 128, 96, 96])
Instance Norm shape: torch.Size([2, 128, 96, 96])
Faltten shape: torch.Size([2, 9216, 64])
torch.Size([2, 64, 96, 96]) cuda:0


In [3]:
import math

from einops import rearrange
from torch import Tensor, nn
import torch

from zeta.nn.modules.u_mamba import UMambaBlock

In [13]:
import math

from einops import rearrange
from torch import Tensor, nn

from zeta.nn.modules.simple_mamba import MambaBlock


class UMambaBlock(nn.Module):
    """
    UMambaBlock is a 5d Mamba block that can be used as a building block for a 5d visual model
    From the paper: https://arxiv.org/pdf/2401.04722.pdf

    Args:
        dim (int): The input dimension.
        dim_inner (Optional[int]): The inner dimension. If not provided, it is set to dim * expand.
        depth (int): The depth of the Mamba block.
        d_state (int): The state dimension. Default is 16.
        expand (int): The expansion factor. Default is 2.
        dt_rank (Union[int, str]): The rank of the temporal difference (Δ) tensor. Default is "auto".
        d_conv (int): The dimension of the convolutional kernel. Default is 4.
        conv_bias (bool): Whether to include bias in the convolutional layer. Default is True.
        bias (bool): Whether to include bias in the linear layers. Default is False.

    Examples::
        import torch
        # img:         B, C, H, W, D
        img_tensor = torch.randn(1, 64, 10, 10, 10)

        # Initialize Mamba block
        block = UMambaBlock(dim=64, depth=1)

        # Forward pass
        y = block(img_tensor)
        print(y.shape)

    """

    def __init__(
        self,
        dim: int = None,
        depth: int = 5,
        d_state: int = 16,
        expand: int = 2,
        d_conv: int = 4,
        conv_bias: bool = True,
        bias: bool = False,
    ):
        super().__init__()
        self.dim = dim
        self.depth = depth
        self.d_state = d_state
        self.expand = expand
        self.d_conv = d_conv
        self.conv_bias = conv_bias
        self.bias = bias

        # If dt_rank is not provided, set it to ceil(dim / d_state)
        dt_rank = math.ceil(self.dim / 16)
        self.dt_rank = dt_rank

        # If dim_inner is not provided, set it to dim * expand
        dim_inner = dim * expand
        self.dim_inner = dim_inner

        # If dim_inner is not provided, set it to dim * expand
        self.in_proj = nn.Linear(dim, dim_inner, bias=False)
        self.out_proj = nn.Linear(dim_inner, dim, bias=False)

        # Implement 2d convolutional layer
        # 3D depthwise convolution
        self.conv1 = nn.Conv3d(
            in_channels=dim,
            out_channels=dim_inner,
            kernel_size=3,
            padding=1,
            stride=1,
        )

        self.conv2 = nn.Conv3d(
            in_channels=dim_inner,
            out_channels=dim,
            kernel_size=3,
            padding=1,
            stride=1,
        )

        # Init instance normalization
        self.instance_norm = nn.InstanceNorm3d(dim)
        self.instance_norm2 = nn.InstanceNorm3d(dim_inner)

        # Leaky RELU
        self.leaky_relu = nn.LeakyReLU()

        # Layernorm
        self.norm = nn.LayerNorm(dim)

        # Mamba block
        self.mamba = MambaBlock(
            dim=dim,
            depth=depth,
            d_state=d_state,
            expand=expand,
            d_conv=d_conv,
            conv_bias=conv_bias,
            bias=bias,
        )

    def forward(self, x: Tensor):
        """
        B, C, H, W, D
        """
        b, c, h, w, d = x.shape
        input = x
        print(f"Input shape: {x.shape}")

        # Apply convolution
        x = self.conv1(x)
        print(f"Conv1 shape: {x.shape}")

        # # Instance Normalization
        x = self.instance_norm(x) + self.leaky_relu(x)
        print(f"Instance Norm shape: {x.shape}")

        # TODO: Add another residual connection here

        x = self.conv2(x)

        x = self.instance_norm(x) + self.leaky_relu(x)

        x = x + input

        # # Flatten to B, L, C
        x = rearrange(x, "b c h w d -> b (h w d) c")
        print(f"Faltten shape: {x.shape}")
        x = self.norm(x)

        # Maybe use a mamba block here then reshape back to B, C, H, W, D
        x = self.mamba(x)

        # Reshape back to B, C, H, W, D
        x = rearrange(x, "b (h w d) c -> b c h w d", h=h, w=w, d=d)

        return x
    

import torch
        # img:         B, C, H, W, D
img_tensor = torch.randn(1, 64, 96, 96, 96)

# Initialize Mamba block
block = UMambaBlock(dim=64, depth=1)

# Forward pass
y = block(img_tensor)
print(y.shape)

Input shape: torch.Size([1, 64, 96, 96, 96])
Conv1 shape: torch.Size([1, 128, 96, 96, 96])
Instance Norm shape: torch.Size([1, 128, 96, 96, 96])
Faltten shape: torch.Size([1, 884736, 64])
torch.Size([1, 64, 96, 96, 96])


In [23]:
from __future__ import annotations

import math

import torch
import torch.nn.functional as F
from einops import einsum, rearrange, repeat
from torch import Tensor, nn

from zeta.nn.modules.rms_norm import RMSNorm
from zeta.utils import exists


class MambaBlock(nn.Module):
    """
    Initialize a single Mamba block.

    Args:
        dim (int): The input dimension.
        dim_inner (Optional[int]): The inner dimension. If not provided, it is set to dim * expand.
        depth (int): The depth of the Mamba block.
        d_state (int): The state dimension. Default is 16.
        expand (int): The expansion factor. Default is 2.
        dt_rank (Union[int, str]): The rank of the temporal difference (Δ) tensor. Default is "auto".
        d_conv (int): The dimension of the convolutional kernel. Default is 4.
        conv_bias (bool): Whether to include bias in the convolutional layer. Default is True.
        bias (bool): Whether to include bias in the linear layers. Default is False.

    Examples:
        >>> import torch
        >>> from zeta.nn.modules.simple_mamba import MambaBlock
        >>> block = MambaBlock(dim=64, depth=1)
        >>> x = torch.randn(1, 10, 64)
        >>> y = block(x)
        >>> y.shape
        torch.Size([1, 10, 64])
    """

    def __init__(
        self,
        dim: int = None,
        depth: int = 5,
        d_state: int = 16,
        expand: int = 2,
        d_conv: int = 4,
        conv_bias: bool = True,
        bias: bool = False,
    ):
        """A single Mamba block, as described in Figure 3 in Section 3.4 in the Mamba paper [1]."""
        super().__init__()
        self.dim = dim
        self.depth = depth
        self.d_state = d_state
        self.expand = expand
        self.d_conv = d_conv
        self.conv_bias = conv_bias
        self.bias = bias

        # If dt_rank is not provided, set it to ceil(dim / d_state)
        dt_rank = math.ceil(self.dim / 16)
        self.dt_rank = dt_rank

        # If dim_inner is not provided, set it to dim * expand
        dim_inner = dim * expand
        self.dim_inner = dim_inner

        # If dim_inner is not provided, set it to dim * expand
        self.in_proj = nn.Linear(dim, dim_inner * 2, bias=bias)

        self.conv1d = nn.Conv1d(
            in_channels=dim_inner,
            out_channels=dim_inner,
            bias=conv_bias,
            kernel_size=d_conv,
            groups=dim_inner,
            padding=d_conv - 1,
        )

        # x_proj takes in `x` and outputs the input-specific Δ, B, C
        self.x_proj = nn.Linear(
            dim_inner, dt_rank + self.d_state * 2, bias=False
        )

        # dt_proj projects Δ from dt_rank to d_in
        self.dt_proj = nn.Linear(dt_rank, dim_inner, bias=True)

        A = repeat(torch.arange(1, self.d_state + 1), "n -> d n", d=dim_inner)
        self.A_log = nn.Parameter(torch.log(A))
        self.D = nn.Parameter(torch.ones(dim_inner))
        self.out_proj = nn.Linear(dim_inner, dim, bias=bias)

    def forward(self, x: Tensor):
        """Mamba block forward. This looks the same as Figure 3 in Section 3.4 in the Mamba paper [1].

        Args:
            x: shape (b, l, d)    (See Glossary at top for definitions of b, l, d_in, n...)

        Returns:
            output: shape (b, l, d)


        Official Implementation:
            class Mamba, https://github.com/state-spaces/mamba/blob/main/mamba_ssm/modules/mamba_simple.py#L119
            mamba_inner_ref(), https://github.com/state-spaces/mamba/blob/main/mamba_ssm/ops/selective_scan_interface.py#L311

        """
        (b, l, d) = x.shape

        x_and_res = self.in_proj(x)  # shape (b, l, 2 * d_in)
        x_and_res = rearrange(x_and_res, "b l x -> b x l")
        (x, res) = x_and_res.split(
            split_size=[self.dim_inner, self.dim_inner], dim=1
        )

        x = self.conv1d(x)[:, :, :l]
        x = F.silu(x)

        y = self.ssm(x)

        y = y * F.silu(res)

        output = self.out_proj(rearrange(y, "b dim l -> b l dim"))

        return output

    def ssm(self, x: Tensor):
        """Runs the SSM. See:
            - Algorithm 2 in Section 3.2 in the Mamba paper [1]
            - run_SSM(A, B, C, u) in The Annotated S4 [2]

        Args:
            x: shape (b, d_in, l)    (See Glossary at top for definitions of b, l, d_in, n...)

        Returns:
            output: shape (b, d_in, l)

        Official Implementation:
            mamba_inner_ref(), https://github.com/state-spaces/mamba/blob/main/mamba_ssm/ops/selective_scan_interface.py#L311

        """
        (d_in, n) = self.A_log.shape

        # Compute ∆ A B C D, the state space parameters.
        #     A, D are input independent
        #     ∆, B, C are input-dependent (this is a key difference between Mamba and the linear time invariant S4)

        A = -torch.exp(self.A_log.float())  # shape (d_in, n)
        D = self.D.float()

        x_dbl = rearrange(x, "b d l -> b l d")
        x_dbl = self.x_proj(x_dbl)  # (b, l, dt_rank + 2*n)

        (delta, B, C) = x_dbl.split(
            split_size=[self.dt_rank, n, n], dim=-1
        )  # delta: (b, l, dt_rank). B, C: (b, l, n)
        delta = F.softplus(self.dt_proj(delta))  # (b, l, d_in)

        y = self.selective_scan(
            x, delta, A, B, C, D
        )  # This is similar to run_SSM(A, B, C, u) in The Annotated S4 [2]

        return y

    def selective_scan(self, u, delta, A, B, C, D):
        """Does selective scan algorithm. See:
            - Section 2 State Space Models in the Mamba paper [1]
            - Algorithm 2 in Section 3.2 in the Mamba paper [1]
            - run_SSM(A, B, C, u) in The Annotated S4 [2]

        This is the classic discrete state space formula:
            x(t + 1) = Ax(t) + Bu(t)
            y(t)     = Cx(t) + Du(t)
        except B and C (and the step size delta, which is used for discretization) are dependent on the input x(t).

        Args:
            u: shape (b, d_in, l)    (See Glossary at top for definitions of b, l, d_in, n...)
            delta: shape (b, l, d_in)
            A: shape (d_in, n)
            B: shape (b, l, n)
            C: shape (b, l, n)
            D: shape (d_in,)

        Returns:
            output: shape (b, d_in, l)

        Official Implementation:
            selective_scan_ref(), https://github.com/state-spaces/mamba/blob/main/mamba_ssm/ops/selective_scan_interface.py#L86
            Note: I refactored some parts out of `selective_scan_ref` out, so the functionality doesn't match exactly.

        """
        (b, d_in, l) = u.shape
        n = A.shape[1]

        # Discretize continuous parameters (Δ, A, B)  (see Section 2 Equation 4 in the Mamba paper [1])
        # Note that B is parameterized directly
        deltaA = torch.exp(einsum(delta, A, "b l d_in, d_in n -> b d_in l n"))
        deltaB_u = einsum(
            delta, B, u, "b l d_in, b l n, b d_in l -> b d_in l n"
        )

        # Perform selective scan (see scan_SSM() in The Annotated S4 [2])
        x = torch.zeros((b, d_in, n), device=next(self.parameters()).device)
        ys = []
        for i in range(l):
            x = deltaA[:, :, i] * x + deltaB_u[:, :, i]
            y = einsum(x, C[:, i, :], "b d_in n , b n -> b d_in")
            ys.append(y)
        y = torch.stack(ys, dim=2)  # (b d_in l)

        if D is not None:
            y = y + u * rearrange(D, "d_in -> d_in 1")

        return y


class Mamba(nn.Module):
    """Mamba model.

    Args:
        vocab_size (int): The size of the vocabulary.
        dim (int): The input dimension.
        depth (int): The depth of the Mamba block.
        d_state (int): The state dimension. Default is 16.
        expand (int): The expansion factor. Default is 2.
        dt_rank (Union[int, str]): The rank of the temporal difference (Δ) tensor. Default is "auto".
        d_conv (int): The dimension of the convolutional kernel. Default is 4.

    Examples:
    x = torch.randint(0, 16, (1, 64))
    model = Mamba(16, 64, 5, 16)
    out = model(x)
    print(out)
    """

    def __init__(
        self,
        vocab_size: int = None,
        dim: int = None,
        depth: int = 5,
        d_state: int = 16,
        img_dim: int = 64,
        *args,
        **kwargs,
    ):
        """Full Mamba model."""
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, dim)
        self.norm_f = RMSNorm(dim)
        self.lm_head = nn.Linear(dim, vocab_size, bias=False)
        self.lm_head.weight = self.embedding.weight
        self.mamba_layers = nn.ModuleList(
            [
                MambaBlock(
                    dim=dim, depth=depth, d_state=d_state, *args, **kwargs
                )
                for _ in range(depth)
            ]
        )

        # Projection for img
        self.img_proj = nn.Linear(img_dim, dim)

    def forward(
        self,
        x: Tensor,
        context: Tensor = None,
    ):
        """
        Args:
            x (long tensor): shape (b, l)    (See Glossary at top for definitions of b, l, d_in, n...)

        Returns:
            logits: shape (b, l, vocab_size)

        Official Implementation:
            class MambaLMHeadModel, https://github.com/state-spaces/mamba/blob/main/mamba_ssm/models/mixer_seq_simple.py#L173

        """
        x = self.embedding(x)

        if exists(context):
            # Project the image
            projected_img = self.img_proj(context)

            # Concatenate the image and text
            x = torch.cat([x, projected_img], dim=1)

        for layer in self.mamba_layers:
            x = layer(self.norm_f(x)) + x

        x = self.norm_f(x)
        logits = self.lm_head(x)

        return logits
    


torch.Size([1, 64, 64])


ValueError: too many values to unpack (expected 3)